# Isolation by spatial random walks

Here I explore the isolation by resistence [[McRae 2006](); [McRae 2007]()] approximation to expected genetic distances using simulations under the coalescent, following [[Petkova et al. 2016]()]. I also explore a recent development in the spatial statistics literature, which derives the induced covariance under a spatial-temporal random walk that can be implemented as a simulatanous auto-regressive process (SAR) [[Hanks 2016]()]. This framework allows for an analgous concept of resistence distance on directed graphs which could be of use for inference of asymetric migration in natural systems. See the `ref` directory for a non-exhausitve list of other relevent papers on this topic.

# Background

# Simulations

## Imports / Configuration

Import standand libraries as well as some scripts in `../scripts` that have defined helper function to setup the habitat and simulate data under the coalescent given a habitat

In [ ]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

sns.set_style('whitegrid')

mpl.rcParams['font.size'] = 14
mpl.rcParams['figure.figsize'] = 8, 6

from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform
from scipy.sparse.csgraph import laplacian
from scipy.linalg import pinv
import pickle as pkl
import os

# import helper functions for this notebook
import sys
sys.path.append('../scripts/')
from define_habitat import gen_lattice, quadratic_barrier_weights
from simulate_genotypes import stepping_stone

## Setup the habitat

Here we simulate a habitat as a $8 \times 8$ triangular lattice $\mathcal{G}$ consiting of nodes and edges $\{\mathbf{v}, \mathbf{e}\}$. To simulate a barrier we define a quadratic function $f(s)$ of the spatial positions $s$ of each node. $f(s)$ is minimized at the middle of the barrier and the barrier is always set in the center of the habitat. We will consider different migration patterns later.

In [ ]:
n = 8
p = 8
d = n * p
lattice_dict = gen_lattice(n, p)
g = lattice_dict['g']
s = lattice_dict['s']
v = lattice_dict['v']
pos_dict = lattice_dict['pos_dict']
m_min = .01 # max migration level
m_max = 3. # min migration level
g = quadratic_barrier_weights(g, s, m_min, m_max)
weights = [g[i][j]['m'] for i,j in g.edges()]

The triangular lattice is visualized with edge widths proportional to the defined edge weights, note we multiply by some constant just for visualization purposes. We color the nodes based on there position on the x-axis as that is more pertinent for the position of the barrier ...

In [ ]:
nx.draw(g, pos=pos_dict, node_size=200, node_color=s[:,0]**2 + (np.sqrt(d) / 2) * s[:,1], cmap=cm.viridis)  
nx.draw_networkx_edges(g, pos_dict, width=2*np.array(weights))

Extract the migration matrix $\mathbf{M}$ which store all the edge weights in $\mathcal{H}$

In [ ]:
m = nx.adjacency_matrix(g, weight='m')
m = m.toarray()
d = m.shape[0]

As expected we see that the migration matrix $\mathbf{M}$ is extremley sparse as only neighboring nodes are connected

In [ ]:
plt.imshow(m, cmap=cm.viridis)
plt.colorbar()

## Simulate genotypes

Here we simulate genotypes under the coalescent using `msprime` ... this may take a bit of time. Specifically we simulate 10 haploid individuals per node in 5000 indepedent regions of the genome. See `../scripts/simulate_genotypes` for default parameters

In [ ]:
%%time
path = '../output/y_barrier_stepping_stone.pkl'
if os.path.exists(path):
    with open(path, 'rb') as geno:
        y = pkl.load(geno)
else:
    with open(path, 'wb') as geno:
        y = stepping_stone(m, n_rep=5000, n=10)
        pkl.dump(y, geno)

In [ ]:
n, p = y.shape 
print(n, p)

Extract individual positions, demes, and migration rates

In [ ]:
v_obs = np.repeat(v, int(n / d)).T
s_obs = np.vstack([np.repeat(s[:,0], int(n / d)), np.repeat(s[:,1], int(n / d))]).T
m_obs = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        m_obs[i,j] = m[v_obs[i], v_obs[j]]

Here we visualize the site frequency spectrum and can see an enricment of common variants under the netural constant population size / random mating expectation which is expected given the migration model we have defined!

In [ ]:
dac = np.sum(y, axis=0) 
x = np.arange(1, n) / n
sfs = np.histogram(dac, bins=np.arange(1, n+1))[0]
plt.semilogy(x, sfs / sfs[0], '.')
plt.semilogy(x, 1 / (x * n), '--')
plt.xlabel('Derived Allele Frequency')
plt.ylabel('log(Count)')

Lets remove rare variants for subsequent visualization

In [ ]:
fil_idx = np.where(((dac / n) >= .05) & ((dac / n) <= .95))[0]
y = y[:,fil_idx]
p = float(y.shape[1])
print(n, p)

Normalize the data for PCA

In [ ]:
mu = np.mean(y, axis=0)
std = np.std(y, axis=0)
z = (y - mu) / std

Running PCA on the normalized genotype data, we see a strong signature of the barrier.

In [ ]:
pca = PCA(n_components=5)
pca.fit(z.T)
pcs = pca.components_.T
plt.scatter(pcs[:,0], pcs[:,1], c=s_obs[:,0]**2 + (np.sqrt(d) / 2) * s_obs[:,1], cmap=cm.viridis)
plt.xlabel('PC1')
plt.ylabel('PC2')

## Computing distances

Compute the squred euclidian genetic distance using the mean centered genotypes

In [ ]:
d_geno = squareform(pdist((y - mu), metric='seuclidean')) / p

We compute the resistence distance using the inverse of the graph laplacian ...


$$
\mathbf{L} = \mathbf{D} - \mathbf{M}
$$

$$
\mathbf{R} = \mathbf{1}diag(\mathbf{L}^{-1})^T + diag(\mathbf{L}^{-1})\mathbf{1}^T - 2\mathbf{L}^{-1}
$$

where $\mathbf{D}$ is a diagonal matrix storing the degree $d_{ii} = \sum_{k \neq i} d_{ik}$ of each node and $\mathbf{1}$ is a column vector of size $n$. The matrix form above correponds to ...

$$
r_{ij} = l^{-1}_{ii} + l^{-1}_{jj} - 2l^{-1}_{ij}
$$

Additionaly we compute the random-walk distance, following [[Hooten 2016]()] by ...

$$
\tilde{\mathbf{R}} = \mathbf{1}diag\big((\mathbf{L}\mathbf{L}^T)^{-1}\big)^T + diag\big((\mathbf{L}\mathbf{L}^T)^{-1}\big)\mathbf{1}^T - 2(\mathbf{L}\mathbf{L}^T)^{-1}
$$

Compute the geographic distance based on the lattice $\mathcal{G}$ positions

In [ ]:
d_geo = squareform(pdist(s_obs, metric='seuclidean')) / 2

Compute the graph laplacian

In [ ]:
l = laplacian(m_obs)
print(l.shape)

ones = np.ones(n).reshape(n, 1)
print(ones.shape)

We can see the graph laplacian is sparse as $\mathbf{M}$ is sparse. We can think of $\mathbf{L}$ here as a sparse presicion matrix.

In [ ]:
plt.imshow(l, cmap='bwr', norm=mpl.colors.Normalize(vmin=-np.max(l), vmax=np.max(l)))
plt.colorbar()

Compute the resistence distance $\mathbf{R}$

In [ ]:
linv = pinv(l)
linv_diag = np.diag(linv).reshape(n, 1)
d_res = ones.dot(linv_diag.T) + linv_diag.dot(ones.T) - (2. * linv)

Compute the random-walk distance $\tilde{\mathbf{R}}$

In [ ]:
llt = l.dot(l.T)
plt.imshow(llt, cmap='bwr', norm=mpl.colors.Normalize(vmin=-np.max(llt), vmax=np.max(llt)))
plt.colorbar()

In [ ]:
lltinv = pinv(llt)
lltinv_diag = np.diag(lltinv).reshape(n, 1)
d_rw = ones.dot(lltinv_diag.T) + lltinv_diag.dot(ones.T) - (2. * lltinv)

Visualize the semivarigram for difference input distances

In [ ]:
tri_idx = np.tril_indices(n, -1)
dist_df = pd.DataFrame({'Geographic Distance': d_geo[tri_idx], 
                        'Genetic Distance': d_geno[tri_idx],
                        'Resistence Distance': d_res[tri_idx],
                        'Random-Walk Distance': d_rw[tri_idx]
                       })


In [ ]:
print('r2 = {}'.format(np.corrcoef(dist_df['Geographic Distance'], dist_df['Genetic Distance'])[0, 1]))
sns.lmplot(x='Geographic Distance', y='Genetic Distance', data=dist_df)

In [ ]:
print('r2 = {}'.format(np.corrcoef(dist_df['Resistence Distance'], dist_df['Genetic Distance'])[0, 1]))
sns.lmplot(x='Resistence Distance', y='Genetic Distance', data=dist_df)

In [ ]:
print('r2 = {}'.format(np.corrcoef(dist_df['Random-Walk Distance'], dist_df['Genetic Distance'])[0, 1]))
sns.lmplot(x='Random-Walk Distance', y='Genetic Distance', data=dist_df)


We can see that the random-walk distance has the highest correlation with the genetic distance out of the 3 we compare here. I'm confused why the resistence distance is performing so poorly here as it should be better than geographic distance ... perhaps its some bug in the code.